In [ ]:
%pip install transformers torch torchtext scikit-learn pandas
%pip install torch==2.0.1 torchtext==0.15.2
!pip install tensorflow
%pip install transformers torch scikit-learn
!pip install datasets





In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


sentences = [
    "I received a call from someone asking for my bank account details.",
    "You've won a lottery! Send us your bank details to claim the prize.",
    "Please update your account information to prevent deactivation.",
    "Hey, just wanted to check in on our meeting next week.",
    "I need help with my order; it didn't arrive on time.",
    "Your account has been compromised; please send your password to fix it.",

]
labels = [1, 1, 1, 0, 0, 1]


tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=20, padding='post', truncating='post')


X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)


model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=20),
    SpatialDropout1D(0.2),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, np.array(y_train), epochs=20, batch_size=1, validation_data=(X_test, np.array(y_test)))

loss, accuracy = model.evaluate(X_test, np.array(y_test))
print(f"Test Accuracy: {accuracy:.2f}")



In [ ]:
new_sentence = ["z"]
new_sequence = tokenizer.texts_to_sequences(new_sentence)
new_padded = pad_sequences(new_sequence, maxlen=20, padding='post', truncating='post')
prediction = model.predict(new_padded)
print("Fraud probability:", prediction[0][0])